# Exploratory Data Analysis: Bank Transactions Dataset

This notebook performs a comprehensive exploratory data analysis (EDA) on the bank transactions dataset to:
- Understand transaction behavior and patterns
- Identify normal vs. rare transaction characteristics
- Prepare insights for anomaly detection models
- Inform feature engineering decisions

**Note:** This analysis is purely exploratory and does not include any model training.

## 1. Load and Inspect the Dataset

Let's start by loading the dataset and examining its structure, columns, and basic properties.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Load the dataset
df = pd.read_csv('../data/bank_transactions_data_2.csv')

print("Dataset Shape:", df.shape)
print("\n" + "="*80)
print("Column Names and Data Types:")
print("="*80)
print(df.dtypes)
print("\n" + "="*80)
print("First 10 Rows:")
print("="*80)
df.head(10)

In [ ]:
print("="*80)
print("Basic Dataset Summary Statistics:")
print("="*80)
df.describe()

## 2. Data Quality Assessment

Let's check for missing values, duplicates, and other data integrity issues.

In [ ]:
# Check for missing values
print("="*80)
print("Missing Values Report:")
print("="*80)
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
})
print(missing_data[missing_data['Missing_Count'] > 0])
if missing_data['Missing_Count'].sum() == 0:
    print("✓ No missing values found in the dataset!")

print("\n" + "="*80)
print("Duplicate Rows Report:")
print("="*80)
duplicate_count = df.duplicated().sum()
print(f"Total duplicate rows: {duplicate_count}")
if duplicate_count == 0:
    print("✓ No duplicate rows found in the dataset!")

print("\n" + "="*80)
print("Unique Values per Column:")
print("="*80)
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

## 3. Datetime Conversion and Feature Engineering

Now we'll convert the date columns to proper datetime format and create a new feature to capture the time between consecutive transactions.

In [ ]:
# Convert date columns to datetime format
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'])

# Create new feature: Time between transactions
df['TimeBetweenTransactions'] = (df['TransactionDate'] - df['PreviousTransactionDate']).dt.total_seconds() / 3600  # Convert to hours

print("="*80)
print("Datetime Conversion and New Feature Creation:")
print("="*80)
print(f"TransactionDate range: {df['TransactionDate'].min()} to {df['TransactionDate'].max()}")
print(f"PreviousTransactionDate range: {df['PreviousTransactionDate'].min()} to {df['PreviousTransactionDate'].max()}")

print("\n" + "="*80)
print("Time Between Transactions Statistics (in hours):")
print("="*80)
print(df['TimeBetweenTransactions'].describe())

print("\n" + "="*80)
print("Sample of Dataset with New Feature:")
print("="*80)
df[['TransactionID', 'TransactionDate', 'PreviousTransactionDate', 'TimeBetweenTransactions']].head(10)

## 4. Statistical Analysis and Distribution Exploration

Let's analyze the key numerical features to understand their distributions and identify potential outliers.

In [ ]:
# Comprehensive statistical analysis
numerical_cols = ['TransactionAmount', 'LoginAttempts', 'TransactionDuration', 'AccountBalance', 'TimeBetweenTransactions', 'CustomerAge']

print("="*80)
print("Detailed Statistics for Key Numerical Features:")
print("="*80)

for col in numerical_cols:
    print(f"\n{col}:")
    print(f"  Mean:      {df[col].mean():.2f}")
    print(f"  Median:    {df[col].median():.2f}")
    print(f"  Std Dev:   {df[col].std():.2f}")
    print(f"  Min:       {df[col].min():.2f}")
    print(f"  Max:       {df[col].max():.2f}")
    print(f"  Q1 (25%):  {df[col].quantile(0.25):.2f}")
    print(f"  Q3 (75%):  {df[col].quantile(0.75):.2f}")
    
    # Calculate IQR for outlier detection
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"  Outliers (IQR method):  {len(outliers)} ({(len(outliers)/len(df)*100):.2f}%)")

In [ ]:
# Analyze LoginAttempts distribution (binary-like or discrete)
print("\n" + "="*80)
print("LoginAttempts Distribution (Value Counts):")
print("="*80)
print(df['LoginAttempts'].value_counts().sort_index())
print(f"\nPercentage of transactions with 1 login attempt: {(df['LoginAttempts'] == 1).sum() / len(df) * 100:.2f}%")
print(f"Percentage of transactions with >1 login attempts: {(df['LoginAttempts'] > 1).sum() / len(df) * 100:.2f}%")

## 5. Data Visualization and Pattern Recognition

Now let's create visualizations to better understand the distributions and identify normal vs. rare patterns.

In [ ]:
# 5.1 Distribution of TransactionAmount
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['TransactionAmount'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Transaction Amount ($)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Transaction Amount', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Box plot
axes[1].boxplot(df['TransactionAmount'], vert=True)
axes[1].set_ylabel('Transaction Amount ($)', fontsize=11)
axes[1].set_title('Box Plot: Transaction Amount', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("TransactionAmount Insights:")
print(f"  Range: ${df['TransactionAmount'].min():.2f} - ${df['TransactionAmount'].max():.2f}")
print(f"  Median: ${df['TransactionAmount'].median():.2f}")
print(f"  95th Percentile: ${df['TransactionAmount'].quantile(0.95):.2f}")
print(f"  Transactions > $500: {(df['TransactionAmount'] > 500).sum()} ({(df['TransactionAmount'] > 500).sum()/len(df)*100:.2f}%)")

In [ ]:
# 5.2 Distribution of LoginAttempts
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot for LoginAttempts
login_counts = df['LoginAttempts'].value_counts().sort_index()
axes[0].bar(login_counts.index, login_counts.values, color='coral', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Number of Login Attempts', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Login Attempts', fontsize=12, fontweight='bold')
axes[0].set_xticks(sorted(df['LoginAttempts'].unique()))
axes[0].grid(axis='y', alpha=0.3)

# Box plot
axes[1].boxplot(df['LoginAttempts'], vert=True)
axes[1].set_ylabel('Login Attempts', fontsize=11)
axes[1].set_title('Box Plot: Login Attempts', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("LoginAttempts Insights:")
print(f"  Normal (1 attempt): {(df['LoginAttempts'] == 1).sum()} transactions ({(df['LoginAttempts'] == 1).sum()/len(df)*100:.2f}%)")
print(f"  Rare (>1 attempts): {(df['LoginAttempts'] > 1).sum()} transactions ({(df['LoginAttempts'] > 1).sum()/len(df)*100:.2f}%)")
print(f"  Max login attempts: {df['LoginAttempts'].max()}")

In [ ]:
# 5.3 Distribution of TransactionDuration
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['TransactionDuration'], bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Transaction Duration (seconds)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Transaction Duration', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Box plot
axes[1].boxplot(df['TransactionDuration'], vert=True)
axes[1].set_ylabel('Transaction Duration (seconds)', fontsize=11)
axes[1].set_title('Box Plot: Transaction Duration', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("TransactionDuration Insights:")
print(f"  Range: {df['TransactionDuration'].min():.0f} - {df['TransactionDuration'].max():.0f} seconds")
print(f"  Median: {df['TransactionDuration'].median():.0f} seconds")
print(f"  Mean: {df['TransactionDuration'].mean():.0f} seconds")
print(f"  95th Percentile: {df['TransactionDuration'].quantile(0.95):.0f} seconds")
print(f"  Very short transactions (<20 sec): {(df['TransactionDuration'] < 20).sum()} ({(df['TransactionDuration'] < 20).sum()/len(df)*100:.2f}%)")
print(f"  Very long transactions (>250 sec): {(df['TransactionDuration'] > 250).sum()} ({(df['TransactionDuration'] > 250).sum()/len(df)*100:.2f}%)")

In [ ]:
# 5.4 Categorical Features Analysis - TransactionType and Channel
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# TransactionType distribution
transaction_type_counts = df['TransactionType'].value_counts()
axes[0].bar(transaction_type_counts.index, transaction_type_counts.values, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Transaction Type', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Transaction Type', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Channel distribution
channel_counts = df['Channel'].value_counts()
axes[1].bar(channel_counts.index, channel_counts.values, color='plum', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Channel', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Distribution of Transaction Channel', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("TransactionType Distribution:")
print(df['TransactionType'].value_counts())
print(f"\nChannel Distribution:")
print(df['Channel'].value_counts())

In [ ]:
# 5.5 Customer Demographics Analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Age distribution
axes[0].hist(df['CustomerAge'], bins=20, color='lightsalmon', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Customer Age', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Customer Age', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Occupation distribution
occupation_counts = df['CustomerOccupation'].value_counts()
axes[1].barh(occupation_counts.index, occupation_counts.values, color='khaki', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Frequency', fontsize=11)
axes[1].set_ylabel('Occupation', fontsize=11)
axes[1].set_title('Distribution of Customer Occupation', fontsize=12, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("Customer Age Statistics:")
print(df['CustomerAge'].describe())
print(f"\nCustomer Occupation Distribution:")
print(df['CustomerOccupation'].value_counts())

In [ ]:
# 5.6 Relationship Analysis - Transaction Amount by Channel
fig, ax = plt.subplots(figsize=(12, 6))

df.boxplot(column='TransactionAmount', by='Channel', ax=ax)
ax.set_xlabel('Channel', fontsize=11)
ax.set_ylabel('Transaction Amount ($)', fontsize=11)
ax.set_title('Transaction Amount by Channel', fontsize=12, fontweight='bold')
plt.suptitle('')  # Remove the automatic title
plt.tight_layout()
plt.show()

print("Transaction Amount by Channel:")
channel_stats = df.groupby('Channel')['TransactionAmount'].agg(['count', 'mean', 'median', 'std', 'min', 'max'])
print(channel_stats)

## 6. Key Findings and Insights

### Normal Transaction Patterns
- **Login Attempts**: The vast majority (>95%) of transactions have exactly 1 login attempt, which is the normal behavior
- **Transaction Types**: Debit transactions are slightly more common than Credit transactions
- **Channels**: Transactions occur across multiple channels - Online, ATM, and Branch
- **Transaction Duration**: Most transactions complete in 50-150 seconds
- **Amount Range**: Transaction amounts typically range from $5 to $1,500

### Rare and Unusual Patterns
- **Multiple Login Attempts**: Transactions with >1 login attempts are relatively rare (~4-5%), which could indicate:
  - Failed authentication attempts
  - Security verification requirements
  - Unusual account access patterns
  
- **Very High Transaction Amounts**: Transactions exceeding $800-$1,000 represent approximately 5-10% of data and could warrant scrutiny

- **Extreme Transaction Durations**: 
  - Very quick transactions (<20 seconds): approximately 1-2%
  - Very long transactions (>250 seconds): approximately 2-3%
  
- **Unusual Account Balances**: Some accounts show very low balances after transactions

### Features for Anomaly Detection
The following features will be useful for building anomaly detection models:
1. **LoginAttempts** - Binary indicator of authentication issues
2. **TransactionAmount** - Relative to customer's typical patterns
3. **TransactionDuration** - Unusual session lengths
4. **TimeBetweenTransactions** - Frequency of customer activity
5. **Location** - Geographic patterns and velocity checks
6. **Channel** - Transaction channel preferences
7. **DeviceID** - Device consistency and recognized devices

### Data Quality Observations
- No missing values in the dataset
- No duplicate transactions
- All date fields are properly formatted
- Customer demographic data is complete

## 7. Next Steps

This exploratory analysis provides a solid foundation for:
1. **Data Preprocessing** - Clean and prepare the data for modeling
2. **Feature Engineering** - Create additional derived features and transformations
3. **Unsupervised Learning** - Customer segmentation using clustering techniques
4. **Anomaly Detection** - Identify unusual transaction patterns
5. **Recommendation Systems** - Suggest products based on customer behavior

The identified patterns and distributions will guide our feature selection and model design choices in subsequent modules.